In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

csv_path = "pro_2025.csv"
df = pd.read_csv(csv_path, low_memory=False)
df_cols = set(df.columns.str.lower())
if 'patch' in df.columns:
    df['patch_major'] = df['patch'].astype(str).str.extract(r'^(\d+\.\d+)').fillna(df['patch'].astype(str))
if 'position' in df.columns:
    order_pos = ['TOP','JUNGLE','MID','ADC','SUPPORT']
    df['position'] = df['position'].astype(str).str.upper()
    df['position'] = pd.Categorical(df['position'], categories=order_pos, ordered=True)
plt.figure(figsize=(9,5))
counts = pd.Series(1, index=df.index).groupby(df.index).size()
plt.hist(counts, bins=10)
plt.title("Row Count Snapshot")
plt.tight_layout()


In [ ]:

cands = ['league','event','tournament','split','stage']
key = next((c for c in cands if c in df.columns), None)
if key is not None:
    agg = df.groupby(key).size().reset_index(name='rows').sort_values('rows', ascending=False).head(20)
    plt.figure(figsize=(10,6))
    plt.bar(agg[key].astype(str), agg['rows'])
    plt.title("Rows by Competition")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()


In [ ]:

if {'side','result'}.issubset(df.columns):
    tmp = df[['side','result']].copy()
    tmp['result'] = tmp['result'].astype(str).str.lower().map({'win':1,'loss':0,'1':1,'0':0}).fillna(np.nan)
    agg = tmp.groupby('side')['result'].mean().mul(100).reset_index(name='win_rate').dropna()
    plt.figure(figsize=(7,5))
    plt.bar(agg['side'].astype(str), agg['win_rate'])
    plt.title("Win Rate by Side (%)")
    plt.tight_layout()


In [ ]:

if 'gameid' in df.columns:
    by_patch = None
    if 'patch_major' in df.columns:
        by_patch = df.groupby('patch_major')['gameid'].nunique().reset_index(name='games').sort_values('patch_major')
    elif 'patch' in df.columns:
        by_patch = df.groupby('patch')['gameid'].nunique().reset_index(name='games').sort_values('patch')
    if by_patch is not None:
        plt.figure(figsize=(10,5))
        plt.plot(by_patch.iloc[:,0].astype(str), by_patch['games'], marker='o')
        plt.title("Games by Patch")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()


In [ ]:

cand_dur = next((c for c in ['gameduration','game_time','game_time_s','duration','time'] if c in df.columns), None)
if cand_dur is not None:
    v = df[cand_dur]
    v = v / 60.0 if v.max() > 2000 else v
    plt.figure(figsize=(9,5))
    plt.hist(v.dropna(), bins=30)
    plt.title("Game Duration (minutes)")
    plt.tight_layout()


In [ ]:

if {'position','kills','deaths','assists'}.issubset(df.columns):
    safe = df['deaths'].replace(0, np.nan)
    kda = (df['kills'] + df['assists']) / safe
    kda = kda.replace([np.inf,-np.inf], np.nan).fillna(df['kills'] + df['assists'])
    agg = pd.DataFrame({'position':df['position'], 'kda':kda}).groupby('position')['kda'].median().reset_index()
    plt.figure(figsize=(9,5))
    plt.plot(agg['position'].astype(str), agg['kda'], marker='o')
    plt.title("Median KDA by Position")
    plt.tight_layout()


In [ ]:

cand_cspm = next((c for c in ['cspm','cs_per_minute','csmin','cs/min'] if c in df.columns), None)
if cand_cspm is not None and 'position' in df.columns:
    agg = df.groupby('position')[cand_cspm].mean().reset_index()
    plt.figure(figsize=(9,5))
    plt.plot(agg['position'].astype(str), agg[cand_cspm], marker='o')
    plt.title("CS per Minute by Position (mean)")
    plt.tight_layout()


In [ ]:

cand_gdiff15 = next((c for c in ['gldiffat15','golddiffat15','gdiff15','goldat15diff','gd15'] if c in df.columns), None)
if cand_gdiff15 is not None and 'side' in df.columns:
    plt.figure(figsize=(9,5))
    for s in sorted(df['side'].dropna().astype(str).unique()[:5]):
        vs = df.loc[df['side'].astype(str)==s, cand_gdiff15].dropna()
        if len(vs) > 0:
            plt.hist(vs, bins=30, alpha=0.5, label=s)
    plt.title("Gold Diff @15 by Side")
    plt.legend()
    plt.tight_layout()


In [ ]:

champ_col = next((c for c in ['champion','championname','champ'] if c in df.columns), None)
if champ_col is not None:
    pick = df[champ_col].value_counts().head(20).sort_values(ascending=True)
    plt.figure(figsize=(8,8))
    plt.barh(pick.index.astype(str), pick.values)
    plt.title("Champion Pick Count (Top 20)")
    plt.tight_layout()


In [ ]:

ban_cols = [c for c in df.columns if str(c).lower().startswith('ban')]
if ban_cols:
    bans = pd.Series([], dtype=object)
    for c in ban_cols:
        bans = pd.concat([bans, df[c].dropna().astype(str)])
    top = bans.value_counts().head(20).sort_values(ascending=True)
    plt.figure(figsize=(8,8))
    plt.barh(top.index.astype(str), top.values)
    plt.title("Champion Ban Count (Top 20)")
    plt.tight_layout()


In [ ]:

champ_col = next((c for c in ['champion','championname','champ'] if c in df.columns), None)
ban_cols = [c for c in df.columns if str(c).lower().startswith('ban')]
if champ_col is not None:
    picks = df[champ_col].dropna().astype(str)
    if ban_cols:
        bans = pd.Series([], dtype=object)
        for c in ban_cols:
            bans = pd.concat([bans, df[c].dropna().astype(str)])
        pres = pd.concat([picks, bans]).value_counts().head(20).sort_values(ascending=True)
    else:
        pres = picks.value_counts().head(20).sort_values(ascending=True)
    plt.figure(figsize=(8,8))
    plt.barh(pres.index.astype(str), pres.values)
    plt.title("Champion Presence (Pick+Ban) Top 20")
    plt.tight_layout()


In [ ]:

champ_col = next((c for c in ['champion','championname','champ'] if c in df.columns), None)
if champ_col is not None and {'result',champ_col}.issubset(df.columns):
    tmp = df[[champ_col,'result']].copy()
    tmp['result'] = tmp['result'].astype(str).str.lower().map({'win':1,'loss':0,'1':1,'0':0}).fillna(np.nan)
    agg = tmp.groupby(champ_col).agg(picks=('result','size'), wr=('result','mean')).dropna()
    agg['wr'] = agg['wr']*100.0
    top = agg.sort_values('picks', ascending=False).head(30)
    plt.figure(figsize=(9,6))
    plt.scatter(top['picks'], top['wr'])
    for i, (name, r) in enumerate(top.iterrows()):
        plt.annotate(str(name), (r['picks'], r['wr']))
    plt.title("Champion Win Rate vs Picks (Top 30 picks)")
    plt.xlabel("Picks")
    plt.ylabel("Win Rate (%)")
    plt.tight_layout()


In [ ]:

obj_cols = [c for c in ['dragons','barons','towers','heralds','elderdragons'] if c in df.columns]
team_key = next((c for c in ['team','teamname'] if c in df.columns), None)
if obj_cols and team_key is not None:
    agg = df.groupby(team_key)[obj_cols].mean().reset_index()
    for c in obj_cols:
        top = agg.nlargest(15, c)[[team_key, c]].iloc[::-1]
        plt.figure(figsize=(9,6))
        plt.barh(top[team_key].astype(str), top[c])
        plt.title(f"{c} per Game (Top 15 teams)")
        plt.tight_layout()
        break


In [ ]:

if 'patch_major' in df.columns:
    counts = df['patch_major'].value_counts().sort_index()
    plt.figure(figsize=(10,5))
    plt.plot(counts.index.astype(str), counts.values, marker='o')
    plt.title("Patch Distribution")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
